- TODO:
    - Add color when there is sentence in `== ... ==`
    - Fix image dimension according to google chrome/github
    - Fix image display on github (check reserch-paper-summary repo)

# I/ Seq2Seq without attention

> - Source: 
>   - [Language Modeling - Lena Volta](https://lena-voita.github.io/nlp_course/language_modeling)
>   - [Seq2Seq & Attention - Lena Volta](https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html)

- Let's first understand how to model language using Neural Network. Intuitively, neural Language Models needs do two things:
    - **process context**: The main idea here is to get a vector representation of the input. This part could be different depending on model architecture (e.g., RNN, CNN, whatever you want), but the main point is to encode context.
    - **generate a probability distribution for the next token**: Once a context has been encoded (vector representation), a model uses it to predict a probability distribution for the next token - see below.
    <p align="center"> <img src="./assets/language_modeling.png" height="500" width="1100" /></p> 

- In order to transform our context vector (which suppose to represent the input `I saw a cat on a`) into a probability distribution, we use a linear layer as projection.
<p align="center"> <img src="./assets/linear_layer_projection.png" height="300" width="1200" /></p> 

-  If we zoom a little bit into the linear layer, we noticed that it is performing a matrix multiplication (vectorized dot product !). Thus, what we are actually doing is propagating/weighting the context vector to each class.
<table><tr>
    <td><img src="./assets/linear_layer_zoom.png" height="350" width="800" /></td>
    <td><img src="./assets/linear_layer_viz.png" height="350" width="1200" /></td>
</tr></table>

- Neural classifiers are trained to predict probability distributions over classes. Intuitively, at each step we maximize the probability a model assigns to the correct class using a cross entropy loss.
- For each input sentence (i.e `The cat is the worst <eos>`), we pass it to the NN to get context vector which is then pass to linear layer to get a probability vector (using softmax). 
- The cross entropy loss will be used to compare probability vector with the correct label (represented as one-hot-encoding vector). It will we maximize the probability a model assigns to the correct class and at the same time, minimize sum of the probabilities of incorrect classes (**since sum of all probabilities is constant, by increasing one probability we decrease sum of all the rest**)

<p align="center"> <video controls src="./assets/rnn_lm_training_with_target.mp4" height="350" width="800" /> </p>

---
- The "Neural Network" is in fact an Encoder-Decoder architecture
- Encoder-decoder is the standard modeling paradigm for sequence-to-sequence tasks which consists of 2 components:
    - **Encoder**: Processes the input sequence and compresses the information into a **context vector/sentence embedding/"thought" vector** of a fixed length which is ==expected to be a good summary of the meaning of the whole source sequence.==
    - **Decoder**: Use the last state of the encoder network as the decoder initial state to emit the transformed output. <p align="center"> <img src="./assets/encoder-decoder.png" height="400" width="550" /></p> 

- More precisely, we want to predict a probability distribution for the next token from the decoder context vector.
    <p align="center"> <img src="./assets/encoder-decoder-detailed.png" height="400" width="600" /></p> 

> - The notation `P(* | I saw a cat, <russian sentence>)` is quite weird, lets demystify it:
>   - We train the model to predict probability distributions of the next token based on "source tokens" and "previous target tokens".
> - Thus, `P(* | I saw a cat, <russian sentence>)` is the **probability to get the next word knowing** `I saw a cat` **and** `<russian sentence>`
> - In the image below, `P(*| I saw a, <russian sentence>)` => **probability to get the next word knowing** `I saw a` **and** `<russian sentence>`
> <p align="center"> <img src="./assets/encoder-decoder-russian.png" height="400" width="800" /></p> 

- The whole process looks like this:  <p align="center"> <video controls src="./assets/seq2seq_training_with_target.mp4" height="350" width="800" /> </p>

---

- Now let's define **Seq2Seq**:
    - **Goal**: aims to transform an input (source) sequence to a new one (target sequence). Both sequences can be of arbitrary lengths.
    - **Structure**: encoder-decoder architecture composed of:
        - **Encoder**: Processes the input sequence and compresses the information into a **context vector/sentence embedding/"thought" vector** of a fixed length which is ==expected to be a good summary of the meaning of the whole source sequence.==
        - **Decoder**: Use the last state of the encoder network as the decoder initial state to emit the transformed output.
        - Both the encoder and decoder are RNN (i.e. using LSTM or GRU units) or CNN
        <p align="center"> <img src="./assets/rnn-context-vector-lily.png" height="400" width="800" /></p> 

- Let's code a simple encoder and decoder (using RNN)

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    pairs = pairs[:100]
    print("Sampled %s sentence pairs (for faster training)" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)

    dataset = {}
    dataset["input_lang"] = input_lang
    dataset["output_lang"] = output_lang
    dataset["pairs"] = pairs
    return dataset

dataset = prepareData('eng', 'fra', True)
print(random.choice(dataset["pairs"]))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Sampled 100 sentence pairs (for faster training)
Counting words...
Counted words:
fra 92
eng 62
['je suis faineant .', 'i m lazy .']


In [3]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [4]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    last_layer_encoder_hidden_states_foreach_input = torch.zeros(max_length, encoder.hidden_size, device=device)
    last_time_step_encoder_hidden_states = encoder.initHidden()
    loss = 0

    # https://stackoverflow.com/a/48305882/8623609
    # encoder_output: give you the hidden layer outputs of the network for each time-step, but only for the final layer ("top")
    # encoder_hidden: give you the hidden layer outputs of the network for the last time-step only, but for all layers ("last right column")
    for i in range(input_length):
        last_layer_encoder_hidden_states, last_time_step_encoder_hidden_states = encoder(input_tensor[i], last_time_step_encoder_hidden_states)
        last_layer_encoder_hidden_states_foreach_input[i] = last_layer_encoder_hidden_states.squeeze()

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = last_time_step_encoder_hidden_states

    # "Teacher forcing" trick for faster convergence
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for i in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[i])
            decoder_input = target_tensor[i]  # Teacher forcing
    else:
        # Without teacher forcing: use its own predictions as the next input
        for i in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[i])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

def trainIters(encoder, decoder, dataset, n_iters, print_every=100):
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.001)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.001)
    training_pairs = [tensorsFromPair(dataset["input_lang"], dataset["output_lang"], random.choice(dataset["pairs"]))
                      for i in range(n_iters)]

    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(f"Step: {iter} ({iter / n_iters * 100}%) Loss: {print_loss_avg}")

hidden_size = 256
encoder1 = EncoderRNN(dataset["input_lang"].n_words, hidden_size).to(device)
attn_decoder1 = DecoderRNN(hidden_size, dataset["output_lang"].n_words).to(device)

trainIters(encoder1, attn_decoder1, dataset, 1000)

Step: 100 (10.0%) Loss: 1.4665349586804708
Step: 200 (20.0%) Loss: 0.9519444440205896
Step: 300 (30.0%) Loss: 0.7524634140531221
Step: 400 (40.0%) Loss: 0.6167872877915701
Step: 500 (50.0%) Loss: 0.45494514699776945
Step: 600 (60.0%) Loss: 0.34018826515475903
Step: 700 (70.0%) Loss: 0.29401343563199056
Step: 800 (80.0%) Loss: 0.20605764707426233
Step: 900 (90.0%) Loss: 0.13313032945990558
Step: 1000 (100.0%) Loss: 0.09674693930273254


In [5]:
def evaluate(encoder, decoder, dataset, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(dataset["input_lang"], sentence)
        input_length = input_tensor.size()[0]

        last_time_step_encoder_hidden_states = encoder.initHidden()

        for i in range(input_length):
            last_layer_encoder_hidden_states, last_time_step_encoder_hidden_states = encoder(input_tensor[i], last_time_step_encoder_hidden_states)

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = last_time_step_encoder_hidden_states

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(dataset["output_lang"].index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words[:di + 1]

def evaluateRandomly(encoder, decoder, dataset, n=3):
    for i in range(n):
        pair = random.choice(dataset["pairs"])
        print("Input: ", pair[0])
        print("Expected:", pair[1])
        output_words = evaluate(encoder, decoder, dataset, pair[0])
        output_sentence = ' '.join(output_words)
        print("Pred: ", output_sentence)
        print('')

evaluateRandomly(encoder1, attn_decoder1, dataset)

Input:  je pars maintenant .
Expected: i m going .
Pred:  i m going . <EOS>

Input:  je suis faineant .
Expected: i am lazy .
Pred:  i am lazy . <EOS>

Input:  je suis triste .
Expected: i m sad .
Pred:  i m sad . <EOS>



- **What's wrong with Seq2Seq ?** We are expecting the fixed-length context vector of the last encoder to be a good summarize of the whole input source is too much to ask. Often it has forgotten the first part once it completes processing the whole input.
- ==**The attention mechanism was born (Bahdanau et al., 2015) to resolve this problem**==
- Summary: <p align="center"> <img src="assets/part1-summary.png" height="400" width="500" /></p> 